In [139]:
import requests
from multiprocessing import Process, Manager
import json
import pandas as pd

In [140]:
def get_story(story_id, stories):
    url = 'https://hacker-news.firebaseio.com/v0/item/%d.json' % story_id
    resp = requests.get(url)
    stories.append(resp.json())

In [141]:
def get_top_stories():
    manager = Manager()
    stories = manager.list()
    url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    ids = requests.get(url)
    processes = [Process(target=get_story, args=(sid, stories))
                 for sid in ids.json()[:40]]
    for p in processes:
        p.start()
    for p in processes:
        p.join()
    return stories

In [142]:
df = pd.read_json('../../data/hn.json')

In [143]:
df['id'] = df.index.astype(int)

In [128]:
top_stories = get_top_stories()

In [133]:
new_df = pd.DataFrame([t for t in top_stories])

In [134]:
joined = df.append(new_df)

In [135]:
joined.set_index('id').index

Int64Index([11693423, 11701020, 11701768, 11702025, 11702267, 11702705,
            11702707, 11702847, 11702939, 11702961, 11703128, 11703340,
            11703372, 11703389, 11703545, 11703596, 11703609, 11703618,
            11703640, 11703659, 11703831, 11703842, 11703877, 11704107,
            11704216, 11704241, 11704375, 11704472, 11704554, 11704647,
            11704651, 11704797, 11704906, 11704952, 11704956, 11704991,
            11705063, 11705243, 11705340, 11705391, 11705340, 11704906,
            11704956, 11704797, 11705391, 11703545, 11703596, 11705243,
            11702705, 11704375, 11704991, 11703842, 11704472, 11704647,
            11703128, 11702961, 11703340, 11704107, 11693423, 11702847,
            11703389, 11702267, 11704651, 11703659, 11703618, 11701768,
            11703609, 11703831, 11702025, 11703640, 11702939, 11702707,
            11701020, 11703877, 11704216, 11704554, 11703372, 11704952,
            11704241, 11705063],
           dtype='int64', name=

In [136]:
df = joined.drop_duplicates(subset='id', keep='last')

In [138]:
df.set_index('id').to_json('../../data/hn.json')